In [2]:
%load_ext autoreload
%autoreload 2
# %autoawait asyncio

# Download 17-18.05.21

In [3]:
from src.data.download import download_whole_city
from src.settings import DATA_RAW_DIR
from tqdm import tqdm

In [4]:
RESOLUTION = 9

In [5]:
cities = [
    # "Vienna, Austria",
    # "Minsk, Belarus",
    # "Brussels, Belgium",
    # "Sofia, Bulgaria",
    # "Zagreb, Croatia",
    # "Prague, Czech Republic",
    # "Tallinn, Estonia",
    # "Helsinki, Finland",
    # "Paris, France",
    "Berlin, Germany",
    "Reykjavík, Iceland",
    "Dublin, Ireland",
    "Rome, Italy",
    "Nur-Sultan, Kazakhstan",
    "Latvia, Riga",
    "Vilnius, Lithuania",
    "Luxembourg City, Luxembourg",
    "Amsterdam, Netherlands",
    "Oslo, Norway",
    "Warszawa, PL",
    "Kraków, PL",
    "Łódź, PL",
    "Wrocław, PL",
    "Poznań, PL",
    "Gdańsk, PL",
    "Lisbon, Portugal",
    "Moscow, Russia",
    "Belgrade, Serbia",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Madrid, Spain",
    "Stockholm, Sweden",
    "Bern, Switzerland",
    "London, United Kingdom",
    "New York City, USA",
    "Chicago, USA",
    "San Francisco, USA",
]

In [6]:
# run download whole city in 4 threads
from threading import Thread
from queue import Queue

def worker(q):
    while True:
        item = q.get()
        download_whole_city(item, DATA_RAW_DIR)
        q.task_done()

q = Queue()
ts = []
for i in range(1):
    t = Thread(target=worker, args=(q,))
    t.daemon = True
    t.start()
    ts.append(t)

for city in tqdm(cities):
    q.put(city)

for t in ts:
    t.join()

100%|██████████| 28/28 [00:00<00:00, 165176.53it/s]


Berlin, Germany


  0%|          | 0/29 [00:00<?, ?it/s]

Tag: aerialway exists for city: Berlin, Germany
Tag: aeroway exists for city: Berlin, Germany
Tag: amenity exists for city: Berlin, Germany
Tag: barrier exists for city: Berlin, Germany
Tag: boundary exists for city: Berlin, Germany


: 

: 

In [9]:
while q.qsize() > 0:
    pass

# Process dataset, select tags, add h3 indices of selected resolution

## Append h3 indices of selected resolution

In [ ]:
from src.data.make_dataset import add_h3_indices_to_city

In [ ]:
for city in tqdm(cities):
    add_h3_indices_to_city(city, 8)

## Group selected tags in cities

In [ ]:
from src.data.load_data import load_filter
from src.data.make_dataset import group_city_tags

In [ ]:
cities = [
    "Vienna, Austria",
    "Minsk, Belarus",
    "Brussels, Belgium",
    "Sofia, Bulgaria",
    "Zagreb, Croatia",
    "Prague, Czech Republic",
    "Tallinn, Estonia",
    "Helsinki, Finland",
    "Paris, France",
    "Berlin, Germany",
    "Reykjavík, Iceland",
    "Dublin, Ireland",
    "Rome, Italy",
    "Nur-Sultan, Kazakhstan",
    "Latvia, Riga",
    "Vilnius, Lithuania",
    "Luxembourg City, Luxembourg",
    "Amsterdam, Netherlands",
    "Oslo, Norway",
    "Warszawa, PL",
    "Kraków, PL",
    "Łódź, PL",
    "Wrocław, PL",
    "Poznań, PL",
    "Gdańsk, PL",
    "Lisbon, Portugal",
    "Moscow, Russia",
    "Belgrade, Serbia",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Madrid, Spain",
    "Stockholm, Sweden",
    "Bern, Switzerland",
    "London, United Kingdom",
    "New York City, USA",
    "Chicago, USA",
    "San Francisco, USA",
]

In [ ]:
SELECTED_TAGS = [
    "aeroway",
    "amenity",
    "building",
    "healthcare",
    "historic",
    "landuse",
    "leisure",
    "military",
    "natural",
    "office",
    "shop",
    "sport",
    "tourism",
    "water",
    "waterway",
]

In [ ]:
key_values_to_drop = {
    "natural": [
        "tree",
        "tree_row",
        "valley",
        'scrub',
        'heath',
        'moor',
        'grassland',
        'fell',
        'bare_rock',
        'scree',
        'shingle',
        'sand',
        'wetland',
        'glacier',
        'reef',
        'spring',
        'hot_spring',
        'geyser',
        'peak',
        'volcano',
        'peninsula',
        'isthmus',
        'ridge',
        'arete',
        'cliff',
        'saddle',
        'dune',
        'rock',
        'stone',
        'sinkhole',
        'cave_entrance',
    ],
    "amenity": [
        'bench',
        'grit_bin',
        'parking_space',
        'post_box',
        'sanitary_dump_station',
        'recycling',
        'waste_basket',
        'waste_disposal',
        'waste_transfer_station',
    ],
    "landuse": [
        "grass"
    ]
}

In [ ]:
TAG_FILTER = load_filter("from_wiki.json")

In [ ]:
for city in tqdm(cities):
    group_city_tags(city, 8, filter_values=TAG_FILTER, fill_missing=True)

# Group all cities

In [ ]:
from src.data.make_dataset import group_cities

df = group_cities(cities, RESOLUTION)


In [ ]:
df['city'].value_counts()

In [ ]:
from keplergl import KeplerGl
KeplerGl(data={"cities": df})

In [ ]:
city_buildings = df[['city', *[c for c in df.columns if c.startswith("building")]]].groupby(['city']).sum()

In [ ]:
city_buildings.reset_index().set_index('city')

In [ ]:
city_buildings_percent = city_buildings.div(city_buildings.sum(axis=1), axis=0).reset_index()
city_buildings_percent

In [ ]:
melted = city_buildings_percent.melt(id_vars = 'city').sort_values(by=['city', 'value'], ascending=[True, False])
melted

In [ ]:
melted[melted['variable'] == 'building_yes']

In [ ]:
szczecin = df[df['city']=='Szczecin, PL'][['h3', *[c for c in df.columns if c.startswith('building')]]]
KeplerGl(data={'szczecin': szczecin})

In [ ]:
szczecin.drop(columns='h3', ).melt().sort_values('value', ascending=False)